In [ ]:
## Look at comparison between 2018 and 2024 

## Check with Income / child poverty 


## compare with above and below average 

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import openpyxl



In [ ]:

# Load the data separately 2018 and 2024 
df_2018 = pd.read_csv('data/GCSE_data_English language_2018_Aged 17 to 19_4 and above_Grade outcomes.csv')
df_2024 = pd.read_csv('data/GCSE_data_English language_2024_Aged 17 to 19_4 and above_Grade outcomes.csv')

# Create a column in each dataframe with difference between county average and national ( last column)
df_2018['Difference_from_England_average'] = df_2018.iloc[:, -2] - df_2018.iloc[:, -1]
df_2024['Difference_from_England_average'] = df_2024.iloc[:, -2] - df_2024.iloc[:, -1]
 
## Merge dataframe on column County
df_merged_2018_vs_2024 = pd.merge(df_2018, df_2024, on='County', suffixes=('_2018', '_2024'))
# Check the first few rows of the merged dataframe
#print(df_merged_2018_vs_2024.head())


## import this data data/children-in-low-income-families-local-area-statistics-2014-to-2024 (1).xlsx from this worksheet 8_Absolute_Ward
df_children = pd.read_excel('data/children-in-low-income-families-local-area-statistics-2014-to-2024 (1).xlsx', sheet_name='8_Absolute_Ward', header=9)
# remove rows 1 to 9 and 11
 
# Check the first few rows of the dataframe
print(df_children.columns)


In [ ]:


# Filter relevant columns
cols_needed = [
    'Local Authority [note 2]',
    'Number of children \nFYE 2018',
    'Percentage of children \nFYE 2018 \n(%)\n[note 3]',
    'Number of children \nFYE 2024',
    'Percentage of children \nFYE 2024\n(%)\n[note 3]'
]
data = df_children[cols_needed].copy()

print(data.head())

# Clean percentages: remove '%' and convert, dropping invalid rows
for col in [
    'Percentage of children \nFYE 2018 \n(%)\n[note 3]',
    'Percentage of children \nFYE 2024\n(%)\n[note 3]'
]:
    data[col] = data[col].str.rstrip('%')
    data[col] = pd.to_numeric(data[col], errors='coerce')

# Drop rows where conversion failed
data = data.dropna(subset=[
    'Percentage of children \nFYE 2018 \n(%)\n[note 3]',
    'Percentage of children \nFYE 2024\n(%)\n[note 3]'
])



In [ ]:

data["county"] = data["Local Authority [note 2]"].str.split(" - ").str[0]
data["county"] = data["county"].str.replace(" County", "", regex=False)
data["county"] = data["county"].str.replace(" City", "", regex=False)
# Group by County and compute total children and weighted percentage
summary = (
    data.groupby('Local Authority [note 2]', group_keys=False)
    .apply(
        lambda g: pd.Series({
            'Total Children 2018': g['Number of children \nFYE 2018'].sum(),
            'Weighted Percentage 2018': (g['Number of children \nFYE 2018'] * g['Percentage of children \nFYE 2018 \n(%)\n[note 3]']).sum() / g['Number of children \nFYE 2018'].sum(),
            'Total Children 2024': g['Number of children \nFYE 2024'].sum(),
            'Weighted Percentage 2024': (g['Number of children \nFYE 2024'] * g['Percentage of children \nFYE 2024\n(%)\n[note 3]']).sum() / g['Number of children \nFYE 2024'].sum(),
        })
    )
)


# Display the result
print(summary)
